In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import json
import random
import signal
from collections import OrderedDict
from dataclasses import dataclass
import copy
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import numpy as np
import subprocess
from operator import itemgetter, attrgetter
import re
import subprocess
import cv2
import traceback
import math
import open3d as o3d
import os
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:

def rw_json_data(rw_mode, path, data):
    try:
        if rw_mode == READ:
            with open(path, 'r', encoding="utf-8") as rdata:
                json_data = json.load(rdata)
            return json_data
        elif rw_mode == WRITE:
            with open(path, 'w', encoding="utf-8") as wdata:
                json.dump(data, wdata, ensure_ascii=False, indent="\t")
        else:
            print('not support mode')
    except:
        print('exception')
        return ERROR


def rw_file_storage(rw_cmd, name, left_map, right_map):
    if rw_cmd == WRITE:
        print("WRITE parameters ......")
        cv_file = cv2.FileStorage(name, cv2.FILE_STORAGE_WRITE)
        cv_file.write("Left_Stereo_Map_x", left_map[0])
        cv_file.write("Left_Stereo_Map_y", left_map[1])
        cv_file.write("Right_Stereo_Map_x", right_map[0])
        cv_file.write("Right_Stereo_Map_y", right_map[1])
        cv_file.release()
    else:
        print("READ parameters ......")
        try:
            # FILE_STORAGE_READ
            cv_file = cv2.FileStorage(name, cv2.FILE_STORAGE_READ)
            # note we also have to specify the type to retrieve other wise we only get a
            # FileNode object back instead of a matrix
            left_map = (cv_file.getNode("Left_Stereo_Map_x").mat(), cv_file.getNode("Left_Stereo_Map_y").mat())
            right_map = (cv_file.getNode("Right_Stereo_Map_x").mat(), cv_file.getNode("Right_Stereo_Map_y").mat())

            cv_file.release()

            return DONE, left_map, right_map
        except:
            traceback.print_exc()
            return ERROR, NOT_SET, NOT_SET


def Rotate(src, degrees):
    if degrees == 90:
        dst = cv2.transpose(src)
        dst = cv2.flip(dst, 1)

    elif degrees == 180:
        dst = cv2.flip(src, -1)

    elif degrees == 270:
        dst = cv2.transpose(src)
        dst = cv2.flip(dst, 0)
    else:
        dst = NOT_SET
    return dst


CAM_1 = ["/dev/video4", "imgL"]
CAM_2 = ["/dev/video5", "imgR"]

CAP_PROP_FRAME_WIDTH = 1920
CAP_PROP_FRAME_HEIGHT = 1080

# Defining the dimensions of checkerboard
CHECKERBOARD = (7, 4)
# Termination criteria for refining the detected corners
CRITERIA = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

ENABLE = 1
DISABLE = 0
DONE = 'DONE'
NOT_SET = 'NOT_SET'
READ = 0
WRITE = 1
ERROR = -1
SUCCESS = 1

JSON_FILE = 'stereo_json'
EXTERNAL_TOOL_CALIBRATION = 'calibration_json'
RECTIFY_MAP = "improved_params2.xml"
CAM_DELAY = 1

USE_EXTERNAL_TOOL_CALIBRAION = DISABLE

data_info_dictionary = {'display': {'left': [CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT],
                                    'right': [CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT]}}



In [4]:
def camera_setting():
    print('start open_camera')

    cap1 = cv2.VideoCapture(CAM_1[0])
    cap1_name = CAM_1[1]
    cap2 = cv2.VideoCapture(CAM_2[0])
    cap2_name = CAM_2[1]

    width = cap1.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap1.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print('cap1 size: %d, %d' % (width, height))
    data_info_dictionary['display']['left'][0] = width
    data_info_dictionary['display']['left'][1] = height

    width = cap2.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap2.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print('cap2 size: %d, %d' % (width, height))
    data_info_dictionary['display']['right'][0] = width
    data_info_dictionary['display']['right'][1] = height

    if not cap1.isOpened() or not cap2.isOpened():
        sys.exit()

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break
        if cv2.waitKey(1) & 0xFF == 27:  # Esc pressed
            break
        imgL = Rotate(frame1, 270)
        imgR = Rotate(frame2, 270)
        view_camera_infos(imgL, f'{cap1_name}', 30, 35)
        cv2.circle(imgL, (int(CAP_PROP_FRAME_WIDTH / 2), int(CAP_PROP_FRAME_HEIGHT / 2)), 2, color=(0, 0, 255),
                   thickness=-1)
        cv2.imshow('left camera', imgL)
        view_camera_infos(imgR, f'{cap2_name}', 30, 35)
        cv2.circle(imgR, (int(CAP_PROP_FRAME_WIDTH / 2), int(CAP_PROP_FRAME_HEIGHT / 2)), 2, color=(0, 0, 255),
                   thickness=-1)
        cv2.imshow("right camera", imgR)

        # alpha = 0.5
        #
        # after_frame = cv2.addWeighted(frame1, alpha, frame2, alpha, 0)
        # cv2.imshow('stereo camera', after_frame)

        cv2.waitKey(CAM_DELAY)

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

In [5]:
def view_camera_infos(frame, text, x, y):
    cv2.putText(frame, text,
                (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), lineType=cv2.LINE_AA)


In [7]:
# 1.
# open camera
camera_setting()

start open_camera
cap1 size: 1080, 1920
cap2 size: 1080, 1920


In [8]:
print('data info dictionary', data_info_dictionary)

data info dictionary {'display': {'left': [1080.0, 1920.0], 'right': [1080.0, 1920.0]}}
